# DevCloud for Edgeを使ったベンチマークの方法を理解する
OpenVINOには多くのサンプルプログラムやデモプログラムが添付されています。サンプルプログラムの多くはOpenVINOでできることを示すための機能デモであったり、OpenVINO APIの使用方法を示すためのサンプルプログラムであり、Performance benchmarkには向きません。  
その中で、benchmark_appというサンプルプログラムはベンチマークを取ることに特化して作られたプログラムです。  
benchmark_appにはさまざまなオプションが指定できますが、オプションを省略した場合でも指定されたハードウエアに最適なオプションを自動で適用してベンチマークを実行してくれるので便利です。
- 指定されたDLアクセラレータに最適なオプションを自動で適用してくれる
- 非同期推論を行い、高スループットが出せる条件で計測できる
- おおよそどんなDLモデルでもベンチマーク可能
 - benchmark_appは入力も出力も気にせず、純粋にDLモデルを処理する時間を計測する
 - 推論結果をパースしない。出力blobがいくつあっても、どんなフォーマットでもOK
 - 入力データを必要としない。入力blobがいくつあっても、入力データが用意できなくてもOK
- Python版とC++版が用意されている

#### ここでは、OpenVINOのベンチマークツール(Python版)を使って基本的なDevCloud for Edgeでのbenchmarkの取り方を学びます

ここでは次のことを学びます:
- benchmark_appを**host server system**上で実行する方法
- ジョブを送信し、benchmark_appを**いづれかのedge computing node**上で実行する方法
- ジョブを送信し、benchmark_appを**特定のedge computing node**上で実行する方法

----
## 1. Development Server上でのbenchmark_appの実行の仕方
ここではPython版のbenchmark_appを使用するのでビルドなどは不要ですぐに実行することができます。(C++版もあります)  
ベンチマークを行う対象のDeep Learningモデルが必要となりますが、ここではSqueezenet1.1をダウンロードして使用します。

### 1.1. benchmark_appをコピーする
`benchmark_app.py`はOpenVINOのインストールディレクトリ内にあります。毎回長いパス名を打つのは手間がかかるので、カレントディレクトリにコピーしてきます。

In [ ]:
!pwd
!cp -r $INTEL_OPENVINO_DIR/deployment_tools/tools/benchmark_tool .
!ls -l benchmark_tool

### 1.2. benchmark_appに必要なPython moduleをインストールする
`reqirements.txt`に記載されているbenchmark_appの依存Pythonモジュールをインストールします。この作業は最初の1回だけ必要です。

In [ ]:
!pip3 install -r benchmark_tool/requirements.txt

### 1.3. Neural network modelをダウンロードする
`squeezenet 1.1`のモデルを、OpenVINO付属の`Model downloader`を使ってダウンロードします。`Model downloader`は指定されたモデルをネットワーク経由でダウンロードしてくれます。ダウンロード可能なモデルのリストは`--print_all`オプションで確認可能です。<br>
ダウンロードされたsqueezenetモデルはCaffe形式のモデルなので**OpenVINOで利用できるIRモデル形式に変換する必要があります。**

In [ ]:
# Squeezenet1.1モデルをダウンロード (Caffe形式)
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1

In [ ]:
# (optional) Model downloaderでダウンロード可能なOMZ DLモデルを一覧表示 (OMZ=Open Model Zoo, OpenVINO model zoo)
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --print_all

### 1.4. ダウンロードしたCaffe形式のsqueezenetモデルをIRモデルに変換する
通常、Caffe, TensorFlow, ONNX, MxNetなどのモデルをIRモデルに変換するには、OpenVINO付属の`Model Optimizer`というツールを使用します。<br>
しかしながら、**`Model downloader`でダウンロードしたモデルに関しては`Model converter`を使うことで簡単にIR形式への変換を行うことが可能です。**`Model converter`は`Model Optimizer`のフロントエンドツールで、内部では適切なオプションをつけて`Model optimizer`を実行しています。

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py --name squeezenet1.1 --precisions FP16

### 1.5. 変換されたIRモデルを確認する
IRモデルは`.xml`と`.bin`で構成されます。`.xml`がトポロジー(グラフ)、`.bin`がウエイト＋バイアスデータです。

In [ ]:
!ls -l public/squeezenet1.1/FP16

### 1.6. benchmark_appをDevelopment serverマシンで実行してみる
まずはEdge inference compute nodeを使用せず、**Development host上でbenchmark_appを実行してみます。**Development serverは今現在皆さんがログインし操作しているマシンです。Development serverはXeonプラットフォームなので内蔵GPUがありません。またVPUやFPGAといったアクセラレータも搭載していませんので、**推論に使用できるデバイスはCPUのみ**となります。  
Development host上で実行する場合ジョブの送信なども不要なので簡単に実行できますし、ジョブがキューで待たされることもありません。ローカルのマシンで普通にOpenVINOのアプリケーションを実行するのと同じように実行可能です。  
実行結果ログの下のほうにパフォーマンスデータが表示されますので確認してください。

benchmark_app options:  
- `-m` specify the IR model file (.xml)
- `-niter` specify the number of inference in the benchmarking

In [ ]:
!python3 benchmark_tool/benchmark_app.py \
        -m public/squeezenet1.1/FP16/squeezenet1.1.xml \
        -niter 100

----
## 2. benchmark_appをEdge inference nodeで実行してみる
ジョブを送信し、先ほど実行したbenchmark_appをedge compute node上で実行してみます。

### 2.1 送信するジョブスクリプトを作成する
ここでは実際に送信するジョブ内容を記述したジョブスクリプト(普通のシェルスクリプト)ファイルを`%%writefile`マジックコマンドで作成します。`%%writefile`マジックコマンドは後に続くセル内の内容をすべてファイルに書き出します。

In [ ]:
%%writefile job.sh
cd ~/devcloud-workshop-jp
pip3 install -r benchmark_tool/requirements.txt
python3 benchmark_tool/benchmark_app.py \
        -m public/squeezenet1.1/FP16/squeezenet1.1.xml \
        -niter 100 \
        $*
echo completed


### 2.2 ジョブスクリプトをEdge computing nodeに送信し、終了を待つ
ここではBasic編でも使ったジョブ終了待ちのためのPythonコードを使用しています。  
また、`qsub`コマンドでedge computing nodeを指定せずにジョブを送信しているので、ベンチマークはedge nodeの「どれか」で実行されます。

In [ ]:
# submit a job
job_id=!qsub job.sh

# generate log file name from job_id
job_num = job_id[0].split('.')[0]
log_file='job.sh.o'+job_num
err_file='job.sh.e'+job_num
print('job_id={}, log_file={}'.format(job_id, log_file))

import time
def waitForJobCompletion(jobNumber):
    print('Waiting for job completion...', end='')
    running=True
    while running:
        time.sleep(1)
        running=False
        status_list=!qstat         # Check job status
        for status in status_list:
            if jobNumber in status:    # if job_num is found in the status list, the job is still running
                running = True
        print(status.split()[4], end='')
    print('...Job {} completed'.format(job_num))    

# wait for the job to complete
waitForJobCompletion(job_num)

### 2.3 ジョブの結果のログファイルを確認する
実行されたノード名が`# Resources:`に、ベンチマークの結果が下のほうにありますのでログファイルをチェックします。

In [ ]:
# 1つ前のセルでlog_fileにログファイル名を入れている。環境変数にセットしてcatから使えるようにする
import os
os.environ['log_file']=log_file

!cat $log_file

----
## 3. コンピュートノードを指定してジョブを実行してみる
先ほどはジョブを実行するエッジコンピュートノードを指定せずにジョブを送信しました。この場合、空いている任意のコンピュートノードでジョブが実行されます。パフォーマンスベンチマークなどを行う際にはジョブを割り当てるコンピュートノードに制限をつけ、特定の構成のコンピュートノードで実行されるようにコントロールする必要があります。  
Edge compute nodeには様々なコンフィグレーションのハードウエアがそろっています。CPUもAtomからXeonまで、各種アクセラレータを積んだノードも用意されています。  
利用可能なコンピュートノードを調べるには**`pbsnodes`**コマンドを使用します。ノード指定に必要なノード名は`properties =`の行に含まれますのでその行だけをフィルターして表示します。ノード名はコンマで区切られて複数の名前が並びます。たとえばノードとして`intel-hd-530`を指定した場合、`intel-hd-530`をpropertiesフィールドに含むいずれかノードでジョブが実行されます。厳密にノードを指定したい場合、より狭い名前を指定する必要があります。  
たとえば`idc001skl`のような名前を指定すると特定のノードで実行することが可能ですが、指定したノードのジョブがたまっている場合、長い時間待たされることがあります。

### 3.1 利用可能なコンピュートノードの一覧を表示する
まずは利用可能なコンピュートノード一覧を表示させてみましょう。`pbsnodes`コマンドを使用します。

In [ ]:
!pbsnodes | grep "properties =" | sort | uniq -c

### 3.2 ノードを指定してジョブを送る
ここでは`qsub`コマンドで具体的な実行ノードを指定してベンチマークジョブを送信しています。また、`-F`オプションで`job.sh`に対するオプションを渡しています。
```
job_id=!qsub -l nodes=1:gold6138 job.sh -F "-d CPU"
```
- `qsub`コマンド
 - `qsub`コマンドでジョブを送るときに、`-l`(limit)オプションをつけることでジョブを実行するノードを制限(=指定)することができます。`-l nodes=1:node_name`のように指定します。たとえば、`skylake`ノードで実行したい場合、`qsub -l nodes=1:skylake job.sh`のように指定します。  
 - また、`-F`オプションをつけることでコマンド引数を渡すことも可能です。この場合、ジョブスクリプトが引数を受け取れるようにしておく必要があることに注意してください。  
 - 今回はジョブスクリプトは先に作成した`job.sh`をそのまま利用します。
- ログファイル名
 -ここでは`qsub`コマンドから返される`job id` (`['27214.v-qsvr-1.devcloud-edge']`のような文字列のリスト)からログファイル名を自動生成させています
- ジョブの終了検出
 - 前回は手動で`qstat`コマンドを何度も実行し、`job.sh`ジョブがなくなるのを確認しましたが、今回はPythonを使って自動で終了を検出するようにしています

#### Note: benchmark_appコマンドの引数について
benchmark_appには様々なオプションが指定可能です。`--help`オプションをつけることで使用可能なオプションを表示できますが、代表的なものを下記に示します。  
ここでは`"-d CPU"`として`CPU`を推論デバイスに指定していますが、**`GPU`, `MYRIAD`など他のデバイスを指定することで様々な条件でベンチマークを行うことが可能です。その際には`qsub`コマンドの`-l`オプションで指定する推論デバイスが利用可能なEdge computing nodeを指定することを忘れないようにしてください。**

|Option|Description|
|:--|:--|
|`-m` PATH_TO_MODEL|推論に使用するIRモデルファイルを指定(`.xml`)|
|`-d` TARGET_DEVICE|推論デバイスを指定。`CPU`, `GPU`, `MYRIAD`, `HDDL`, `HETERO:FPGA,CPU`などを指定可能|
|`-niter` NUMBER_ITERATIONS|実行する推論数。省略すると1分間推論を行う|
|`-nireq` NUMBER_INFER_REQUESTS|同時推論実行数。たとえば4を指定すると同じデバイスに推論要求を同時に4つ投げる。Throughputを上げるためにはデバイス特性に合った同時推論数を指定するのが肝要。省略するとbenchmark_appが自動的に推論デバイスに適切な同時推論数を使用|
|`-b` BATCH_SIZE|バッチ推論数|
|`-i` PATH_TO_INPUT|推論に使用する入力画像ファイルを指定。benchmark_appでは省略可能(入力データなしでもベンチマーク可能)|
|`-pc`|レイヤーごとの詳細実行レポートを表示。レイヤーごとの実行時間も含まれる|

In [ ]:
# submit a job
job_id=!qsub -l nodes=1:gold6138 job.sh -F "-d CPU"

# generate log file name from job_id
print('job_id=', job_id)
job_num = job_id[0].split('.')[0]
log_file='job.sh.o'+job_num
err_file='job.sh.e'+job_num
print('log_file=', log_file)

# wait for the job to complete
waitForJobCompletion(job_num)

### 3.3 ジョブの結果を確認する
前回は単純にログファイル全体を表示しましたが、今回は`grep`を使用して必要な部分だけを見やすく表示させています

In [ ]:
os.environ['log_file']=log_file

!grep '# Resources:'                                  $log_file
!grep '\[ INFO \] Device info'                   -A 3 $log_file
!grep '\[Step 11/11\] Dumping statistics report' -A 4 $log_file

----
ここではbenchmark_appをDevCloudで走らせてベンチマークを取る方法について学びました。
benchmark_appはどんなモデルでも走らせることが可能ですので、独自のDLモデルであってもDevCloudにアップロードし、簡単にベンチマークを取ることが可能です。  
DevCloudのストレージは他のユーザーやインテルの管理者からも参照できないようになっていますので安心して利用することが可能です。

## Next => [OpenVINOを使用するC++プロジェクトの作り方と、DevCloudでの実行方法](./cpp-project.ipynb)